# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

Date List Created
Record num : 32763877
Wall time: 1min 23s


In [3]:
%%time
data['df']['tick']      = (data["df"]['Ask'] + data["df"]['Bid'])/2
data['df']['DateTime_frmt']   = pd.to_datetime(data["df"]['DateTime'], format="%Y%m%d %H:%M:%S")
data['df_ohlc'] = data['df'][['DateTime_frmt', 'tick']]

Wall time: 6.87 s


In [4]:
data['df_ohlc'] = data['df'][['DateTime_frmt', 'tick']]
data['candle_size'] = '15T'

In [5]:
%%time
data = get_ohlc(data)

Wall time: 1.68 s


In [10]:
data['pl_candles'] = 5

### Hammer

In [11]:
data['df_ohlc']['cdl_hammer'] = talib.CDLHAMMER(data['df_ohlc']['open'], data['df_ohlc']['high'], data['df_ohlc']['low'], data['df_ohlc']['close'])
data['df_ohlc']['cdl_hammer'] = data['df_ohlc']['cdl_hammer'].replace({100:1})

data['identified_points'] = data['df_ohlc'][data['df_ohlc']['cdl_hammer'] == 1].index
data['test_type'] = 'up'

data = calculate_total_profit(data)

Total pl            : -0.001925000000000297
Num of Transactions : 7
pl Transactions     : 4
Loss Transactions   : 3


### Engulfing candle

In [12]:
data['df_ohlc']['cdl_engulfing'] = talib.CDLENGULFING(data['df_ohlc']['open'], data['df_ohlc']['high'], data['df_ohlc']['low'], data['df_ohlc']['close'])

In [13]:
data['identified_points'] = data['df_ohlc'][data['df_ohlc']['cdl_engulfing'] == 100].index
data['test_type'] = 'up'

data = calculate_total_profit(data)

Total pl            : -0.14025000000000182
Num of Transactions : 1447
pl Transactions     : 708
Loss Transactions   : 739


In [14]:
data['identified_points'] = data['df_ohlc'][data['df_ohlc']['cdl_engulfing'] == -100].index
data['test_type'] = 'down'

data = calculate_total_profit(data)

Total pl            : -0.1924299999999933
Num of Transactions : 1474
pl Transactions     : 707
Loss Transactions   : 767


sys.exit()

### Merge with DF to visualize Data

data['df']['cdl_hammer'] = data['df']['cdl_hammer'].replace({0:np.nan})
data['df']['cdl_hammer'] = np.where(data['df']['cdl_hammer'] == 100, data['df']['tick'], data['df']['cdl_hammer'])
data = merge_ohlc_data(data)

### Plot

data["plot"] = False
data['plot_type'] = 'file'
data['reduce_plot'] = True
data['plot_start']  = '2022-07-20'
data['plot_stop']   = '2022-07-21'
data['things_to_plot'] = ['tick', 'positions']

plot_graph(data)